In [1]:
import pandas as pd 
import plotly.express as px 
import scipy.stats as stats 
import numpy as np 

In [2]:
df3 = pd.read_csv('11_Data.csv')

1. df3데이터를 이용해 아래와 같은 분석을 하고자 한다. 최근성(Recency)가 가장 좋은 상위 10%의 고객들의 성비와 평균 나이를 계산하시오.

In [6]:
cond1 = df3['Recency'] <= df3['Recency'].quantile(0.1)
df4 = df3.loc[cond1]

In [ ]:
df4['성별'].value_counts() / len(df4['성별'])

In [10]:
df4['나이'].mean()

42.55615942028985

In [ ]:
df4.pivot_table(index='성별',values='나이',aggfunc='mean',margins=True)

- 2. 금액(Monetary)이 높은 1000명의 데이터를 추출하여, '사업자구분'에서 '개인'과 '사업자'간 주기(Frequency)의 평균의 차가 있는지 가설 검정을 수행
   - 
가설 및 결론 포함하여 작성

In [16]:
df5 = df3.sort_values(by='Monetary',ascending=False).head(1000)

In [17]:
# 2-1. 각 집단 별 (개인과 사업자에 따른) 주기의 분포가 정규성을 따르는가? 
# 귀무가설 : 해당 숫자데이터는 정규성을 따른다. / 대립가설 : 해당 숫자데이터는 정규성을 따르지 않는다.
cond1 = df5['사업자구분']=='개인'
cond2 = df5['사업자구분']=='사업자'
print(stats.normaltest(df5.loc[cond1]['Frequency']))
print(stats.normaltest(df5.loc[cond2]['Frequency']))
# P.value (귀무가설이 참일 확률) < 0.05 (유의수준 5%) / 대립가설 참

NormaltestResult(statistic=350.2724240673947, pvalue=8.695799794516361e-77)
NormaltestResult(statistic=849.7666155777041, pvalue=2.9889866444145317e-185)


In [18]:
# 2-2. (정규분포가 아닌 연속형 자료) 각 집단 별 주기의 대표값(중앙값)의 차이가 있는지 검정 
# 귀무 가설 : 각 집단 별 주기의 중앙값의 차이는 없다. 
# 대립 가설 : 각 집단 별 주기의 중앙값의 차이는 있다.
stats.ranksums(df5.loc[cond1]['Frequency'], df5.loc[cond2]['Frequency'])
# P.value > 0.05 / 귀무가설 참 / 각 집단 별 주기의 중앙값의 차이는 없다. 

RanksumsResult(statistic=0.10146465851771255, pvalue=0.9191816111531396)

3. 최근성(Recency)과 금액(Monetary)간의 상관성을 확인하고자 한다. 성별에 따른 최근성과 금액의 산점도를 시각화 하시오. 

In [ ]:
px.scatter(df3, x='Recency', y='Monetary')

In [22]:
# 3-1. 최근성과 금액 간 상관성이 있는지 가설 검정을 수행
# 두 연속형 자료가 정규성을 따르는지 확인 
print(stats.normaltest(df3['Recency']))
print(stats.normaltest(df3['Frequency'])) # P.value < 0.05 / 대립가설 참 / 정규성을 따르지 않는다.  

NormaltestResult(statistic=1003.4450089983651, pvalue=1.2725792844076955e-218)
NormaltestResult(statistic=14087.05803696957, pvalue=0.0)


In [23]:
# 3-2. 정규분포를 따르지 않는 숫자 데이터의 상관성 검정을 수행 
# 귀무가설 : 두 연속형 숫자데이터의 상관성이 없다. 
# 대립가설 : 두 연속형 숫자데이터는 서로 상관성이 있다. 
stats.spearmanr(df3['Recency'], df3['Frequency']) # P.value < 0.05 / 대립가설 참 /  두 연속형 숫자데이터는 서로 상관성이 있다. 
# stats.pearsonr( df[] , df[] )  #  두 연속형 자료가 정규분포를 따르는 경우 

SignificanceResult(statistic=-0.5143547499449463, pvalue=0.0)

In [ ]:
# Corrleation (상관 계수) : 두 연속형 자료가 서로 얼만큼 상관성이 있는가? 
# -1 (음의 상관관계 / 반비례) ~ +1 (양의 상관관계 / 비례) / 0 무상관 
df3[['Recency', 'Frequency']].corr() # Pearson Corr (정규성을 따르는 데이터들에 대한 상관계수를 계산) 
# method='pearson'

In [26]:
df3[['Recency', 'Frequency']].corr(method='spearman') 

,Recency,Frequency
Recency,1.000000,-0.514355
Frequency,-0.514355,1.000000


4. 고객등급 ‘VIP’와 ‘Normal’에 따른 고객 연령의 평균의 차이가 있는지 가설 검정을 수행하고, VIP 고객의 '유입경로' 빈도수가 가장 높은 항목에 대해 Recency와 Frequency, Monetary의 기술 통계량을 계산하시오.
    - 가설 수립 및 결론 도출 까지 진행

In [32]:
# 4-1. 고객 연령의 정규성 검정 
cond1 = df3['고객등급']=='VIP'
cond2 = df3['고객등급']=='Normal'
print(stats.normaltest(df3.loc[cond1]['나이']))
print(stats.normaltest(df3.loc[cond2]['나이'])) # P.value < 0.05 / 대립가설 참 

NormaltestResult(statistic=63.35818345907562, pvalue=1.7456021580554742e-14)
NormaltestResult(statistic=99.86360958512842, pvalue=2.0648699646259737e-22)


In [37]:
# 4-2. (정규성을 따르지 않는 연속형 자료의) 집단 간 평균 비교 
# 귀무가설 : VIP그룹과 Normal 그룹의 나이의 중앙값 차이는 없다. / 나이의 유의미한 차이가 없다. 
# 대립가설 : VIP그룹과 Normal 그룹의 나이의 중앙값 차이는 있다.
stats.ranksums(df3.loc[cond1]['나이'], df3.loc[cond2]['나이'])  # P.value < 0.05 / 대립가설 참 
# df3.pivot_table(index='고객등급',values='나이',aggfunc=['mean','median', 'max','min','std'])

RanksumsResult(statistic=6.460509698899809, pvalue=1.0435090462985479e-10)

In [42]:
df_VIP = df3.loc[cond1]
top_vip = df_VIP['유입경로'].value_counts().index[0]

In [ ]:
cond1 = df_VIP['유입경로']==top_vip
df_VIP.loc[cond1][['Recency','Frequency','Monetary']].describe()

5. 회원가입일과 최종접속일 데이터를 이용해 '접속기간'이라는 파생 변수를 생성하고, '접속기간'이 가장 오래된 100명의 고객 명단을 뽑아 result_vip_member.xlsx로 저장하시오.

In [54]:
def func1(row):
    if pd.isnull(row):
        return row
    return row.split(' ')[0]

df3['접속기간'] = pd.to_datetime(df3['최종접속일'].apply(func1)) - pd.to_datetime(df3['회원 가입일'])

In [58]:
df3.sort_values(by='접속기간', ascending =False).head(100).to_excel('result_vip_member.xlsx')

# 예측적 데이터 분석 (PDA)

- 전통 통계 : 회귀 분석 (추출된 데이터를 모두 이용해 회귀식 구성 -> 검증 : P.value)
- 데이터 마이닝 : 기계 학습 (학습 데이터와 검증데이터를 분할 -> 검증 : Test Data)
- 전통 통계에서 말하는 회귀 분석과 데이터 마이닝에서 말하는 기계학습 다른 매커니즘을 가진다.
- 전통 통계에서 회귀 분석 절차
  - 1. 연속형 자료간의 상관 분석을 실시 : 변수 간 산점도나 상관계수를 확인해 경향성을 파악
  - 2. 목표변수와 설명변수를 선택하여 회귀 모델을 구성 (최소제곱법 [해석학적기법 - 대수적기법])
  - 3. 회귀모델의 적합성을 확인
       - 추정된 표준 오차를 확인 / 결정 계수 (설명력 / R²) 확인
       - 모델 유형의 적합성을 나타내는 ANOVA 분석결과를 확인
  - 4. 회귀 계수를 확인
       - 회귀 계수의 유의성 (P.value)
       - 다중공선성 (Multi-Collinearity) : (설명변수간의 상관관계가 높아 발생하는 문제 -> 상관관계가 높은 변수들 간의 통제할 수 없는 오차를 확인)
  - 5. 오차(실제데이터 - 예측데이터)의 가정 확인
       - 오차의 기본 가정 ( 오차의 정규성, 등분산성, 독립)
  - 6. 모델 선정 

In [ ]:
# 12_Data.csv : 구강 내 체강세포 이미지 데이터 
# 세포를 현미경으로 촬영해 세포가 갖는 기하적 특성을 통계량으로 정리
df1 = pd.read_csv('12_Data.csv') # Diagnosis : M - 암세포 / B - 일반세포
print(df1.shape)
df1.head(2)

In [ ]:
# Mean Concavity -> Y / Mean Compactness -> X 
px.scatter(df1, x='Mean Compactness', y='Mean Concavity', color='Diagnosis')

In [65]:
import statsmodels.api as sm 

In [66]:
# X 값과 Y 값을 설정 
X = df1['Mean Compactness']
Y = df1['Mean Concavity']

In [68]:
# 절편를 계산하기 위한 상수항을 임의로 생성 
X2 = sm.add_constant(X)

In [69]:
# 회귀 모델을 계산 
model = sm.OLS(Y, X2)
result = model.fit()

In [70]:
# 모델 해석 
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Mean Concavity   R-squared:                       0.780
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     2009.
Date:                Thu, 16 May 2024   Prob (F-statistic):          1.62e-188
Time:                        10:43:24   Log-Likelihood:                 1062.9
No. Observations:                 569   AIC:                            -2122.
Df Residuals:                     567   BIC:                            -2113.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0503      0.003    -14.463      0.000      -0.057      -0.043
Mean Compactness     1.3331      0.030     44.823      0.000       1.275       1.391
==============================================================================
Omnibus:                      143.185   Durbin-Watson:                   1.915
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              914.547
Skew:                           0.941   Prob(JB):                    2.56e-199
Kurtosis:                       8.919   Cond. No.                         19.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- R²(R Square, 결정계수) : 0 ~ 1 범위 
  - R2는 데이터의 총 변동 중에서 회귀식에 의해 설명되는 변동의 비율
  - 회귀식이 데이터를 얼마나 잘 설명하고 있는가 / 회귀식이 얼마나 데이터와 가까운지
  - 회귀 모델에 입력변수(X)의 개수가 많을 수록 자연스럽게 증가
    
- adj R² (Adjusted R Square, 수정된 결정 계수
  - 다중회귀분석(X가 여럿있는 회귀식을 구성)에서 데이터 수가 많아지면 결정계수 값이 올라가는 현상을 방지하기 위해 계산

- F-statistic (F-검정 통계량) : 해당 회귀식이 기울기가 존재하고, 회귀선이 특정 경향성을 따르는지에 대한 가설검정 결과 통계량
- Prob (F-statistic): 회귀식의 기울기가 유의미한지 검정한 결과 Pvalue
  - 귀무가설 : 기울기가 없고, 점들이 흩어져 있다.
  - 대립가설 : 기울기가 있고, 점들이 특정 경향을 갖고 있는 상태
  - Prob (F-statistic) < 0.05 / 대립가설 / 회귀식이 유의미하다

- **여러개의 모델끼리 비교할 때 사용**
- Log Likelihood : 통계 모델의 적합도를 나타내는 척도 (AIC와 BIC 계산)
- AIC (이케아케 정보 기준) : 모델의 적합도와 복잡성을 동시에 고려한 통계량 (Log Likelihood으로 부터 계산) / 낮으면 낮을수록 좋음
- BIC (베이지안 정보기준) : 모델의 적합도와 복잡성을 동시에 고려한 통계량 / 낮으면 낮을수록 좋음 (0으로 갈수록)

- Covariance Type :  공분산 행렬에 대한 추측
  - non-robust : 특이값과 이상치에 대해 민감하게 반응하는 모델 (전통적인 최소제곱법을 사용하여 공분산을 추정한다는 의미)
  - robust : 특이값과 이상치에 대해 민감하게 반응하지 않는 모델

- coef : 회귀 계수
  - Y = 1.3331 x Mean Compactness - 0.0503
    
- P >|t| : 해당 회귀 계수값이 유의미한지에 대한 P.value
  - 귀무가설 : 회귀 계수는 0이다. (해당 독립변수는 종속변수에 영향을 미치지 못한다. / 해당 회귀 계수가 회귀식에 유의하지 않음)
  - 대립가설 : 회귀 계수는 0이 아니다. (해당 독립변수는 종속변수에 영향을 주고 있다. / 해당 회귀 계수가 회귀식에 유의하다)
- [0.025 - 0.975] : 회귀 계수의 신뢰구간
  - 신뢰구간 : 현재 표본으로 모집단을 추정했을때, 모집단에 평균값이 포함될 수 있는 구간

- Omnibus(옴니버스 검정) : 회귀 모델의 전반적인 유효성을 검정
  - 모델의 적합성에 대한 종합적인 통계량을 F 통계량으로 확인
  - 옴니버스 검정이 유의하다 : 적어도 하나의 X가 Y에 유의미한 영향을 주고 있다는 의미 ( Prob(Omnibus) < 0.05 )

- Durbin-Watson (더빈왓슨) 통계량 : 회귀모델의 오차항 간의 자기상관성을 판별 통계량 / 오차가 한쪽으로 편향되어 있는지 아닌지를 판별하는 수치
  - 0~4사이의 값을 가지며, 2로 가면 갈 수록 오차의 편향이 없는 상태
  - 다른 여러 모델과 비교했을 때, 가장 2에 가까운 모델을 찾는 형태

- Jarque-Bera (JB 자크베라 검정): 회귀 모델의 오차항이 정규분포를 따르는지 따르지 않는지 검정
  - 귀무가설 : 오차 항의 분포는 정규분포를 따른다 (회귀식이 대부분의 데이터를 잘 설명하고 있다.) 
  - 대립가설 : 오차 항의 분포는 정규분포를 따르지 않는다
  - P.value > 0.05 (귀무가설 참) : 오차 항의 분포는 정규분포를 따른다.
    - 회귀식이 데이터에 대해 안정적인 분포를 취하고 있다. 

# 기계학습에서 머신러닝 

- **지도 학습 절차**
  - 1. 데이터 핸들링 (데이터 불러오기 , 병합 , 재구조화, 파생변수 생성, 이상치 제거)
       - 데이터 구조적 관점에서 학습을 수행할 수 있는 형태로 변환하는 과정
  - 2. 학습의 목표변수 Y와 설명변수 X를 설정 (변수로 선언)
  - 3. 학습 데이터(Train Set)와 검증 데이터(Test Set)를 분할
       - Validation Set 교차검증 과정에서 사용되는 데이터 셋 / 학습에 활용 됌
       - Test Set 학습에 절대 참여하지 않음
  - 4. 학습 수행
       - 특성공학(결측값 처리, 스케일링, 교차검증...)  + 학습 동시 수행
  - 5. 모델 평가
       - 학습 능력 평가 : 알고리즘이 학습 데이터로 적절한 수식을 잘 찾았는가?
       - 일반화 능력 평가 : 학습에 참여하지 않은 (Test Set)데이터로 새로 들어올 데이터에 대해 얼마나 잘 작동하는가?
  - 6. 새로운 데이터 적용  

In [71]:
# 이탈 고객 분류 모델을 구성 
df1 = pd.read_csv('01_Data.csv')

In [73]:
df1['State'].unique()

array(['계약확정', '기간만료', '해약확정', '해약진행중'], dtype=object)

In [75]:
# 1. 데이터 핸들링 
df1['해약여부'] = df1['State'].replace({'계약확정':'정상', '기간만료':'정상', '해약확정':'해약', '해약진행중':'해약'})
df1['해약여부'].value_counts()

해약여부
정상    50665
해약      636
Name: count, dtype: int64

In [76]:
df2 = df1.dropna() # 결측값 제거 

In [78]:
# 2. X와 Y를 선정 
X = df2[['Age', 'Term', 'Amount_Month', 'Credit_Rank']]
Y = df2['해약여부']

In [79]:
# 3. 학습 데이터와 검증 데이터를 분할
# scipy (Science Python) : 응용 통계 분석 및 처리 
# scikit Learn (Science Python Toolkit Learning) : 정형데이터 데이터마이닝 기법  
from sklearn.model_selection import train_test_split 

In [82]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y) # 학습 데이터 검증 데이터 분할 / 75 : 25 
print(X_train.shape , X_test.shape)

(30485, 4) (10162, 4)


In [83]:
# 4. 학습 실시 
from sklearn.tree import DecisionTreeClassifier # 의사결정나무 모델 (선생님)

In [84]:
model = DecisionTreeClassifier() # 학습 알고리즘을 변수로 선언 
model.fit(X_train, Y_train) # fit()  : 학습 데이터에 대한 학습 실시 (수식 도출 실시) 

DecisionTreeClassifier()

In [85]:
# 5. 평가 
from sklearn.metrics import accuracy_score

In [86]:
# 예측 값을 계산 하여 평가 
Y_train_pred = model.predict(X_train) # predict() : 예측을 수행하는 함수 
Y_test_pred = model.predict(X_test)

In [87]:
# 실제값과 예측값을 비교하여 평가 
accuracy_score(Y_train, Y_train_pred) # 학습 능력 

0.9890765950467443

In [88]:
accuracy_score(Y_test, Y_test_pred) # 일반화 능력 

0.9852391261562684

In [ ]:
# 6. 새로운 실제 데이터 입력 
x1 = input('고객 연령을 입력 하시오 : ')
x2 = input('계약 기간(월) 입력 하시오 : ')
x3 = input('계약 금액 (월랜탈 비용) 입력 하시오 : ')
x4 = input('신용등급을 입력 하시오 : ')
input_data = pd.DataFrame(data= [[x1,x2,x3,x4]], columns=['Age', 'Term', 'Amount_Month','Credit_Rank'])
input_data

In [94]:
model.predict(input_data)

array(['정상'], dtype=object)

# 학습 모델 평가 

## 분류 모델 평가

- **정확도 (Accuracy)** = (예측 결과가 동일한 데이터 수)/(전체 예측 데이터 수)
- 클래스가 균일(예1 - 정상 /해약의 비율이 큰 차이가 없는 경우)에 사용하며, 모든 클래스가 동등한 중요도를 가질때
- 이진 분류에서 불균형 데이터(Imbalanced Data)가 오는경우 모델의 성능이 왜곡되어 계산
- **오차행렬 (Confusion Matrix)**

![img1](https://www.researchgate.net/profile/Juan_Banda/publication/256418526/figure/fig1/AS:297921313558528@1448041384565/Confusion-matrix-example.png)

- **정밀도 (Precision)** : True Positive / (True Positive + False Positive)
  - 해약을 기준으로 Precision : (해약을 해약으로 맞춘 수) / (모델이 해약으로 예측한 데이터 수 / 잘 예측한 값 + 잘못 예측한 값)
  - 문제가 없는 데이터를 문제가 있다고 잘못 판단할 때 발생하는 이슈를 나타내는 지표
  -  False Positive가 낮아지는 것에 초점을 맞춘다 / 스팸 메시지 분류기
- **재현율 (Recall)** : True Positive / (True Positive + False Negative)
  - 해약을 기준으로 Recall : (해약을 해약으로 맞춘 수) / (실제 해약인 데이터 / 잘 예측값 + 해약인데 정상으로 잘못 예측된 값)
  - 문제가 있는데 문제가 없다고 잘못 판단 할 때 이슈가 발생하는 경우
  - 민감도(Sensitivity) or TPR(True Positive Rate)
- 두 수치가 좋을수록 좋은 모델이지만, 이진 분류에서는 Recall - Precision Trade off 관계가 있음
- **F1-Score** :정밀도와 재현율을 백분위로 계산한 지표
  - 2x(정밀도 x 재현율) / (정밀도 + 재현율) 

In [95]:
from sklearn.metrics import classification_report

In [96]:
print(classification_report(Y_train, Y_train_pred)) 

              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     30106
          해약       0.93      0.13      0.23       379

    accuracy                           0.99     30485
   macro avg       0.96      0.57      0.61     30485
weighted avg       0.99      0.99      0.99     30485



In [97]:
print(classification_report(Y_test, Y_test_pred)) 

              precision    recall  f1-score   support

          정상       0.99      1.00      0.99     10031
          해약       0.00      0.00      0.00       131

    accuracy                           0.99     10162
   macro avg       0.49      0.50      0.50     10162
weighted avg       0.97      0.99      0.98     10162



# Recall - Precision Trade off 

- Recall-Precision Trade off : 이진 분류에 있어, (불균형 데이터를 분류할 때) 한쪽 지표가 올라가면, 다른 한쪽은 떨어지는 현상
- 정밀도나 재현률을 특별히 강조해야하는 경우, Recall - Precision Trade off 를 활용하여, 정밀도 또는 재현율을 올릴 수 있다.
- Threshold (임계값) : 분류를 함에 있어, 분류확률을 기준으로 클래스를 분류 

In [ ]:
pd.DataFrame(model.predict_proba(X_train), columns=['정상확률', '해약확률']).head(10)

In [103]:
# 위의 분류 결정 임계값를 다르게 주어, 해약할 확률이 10%만 넘어도 "해약"으로 분류 
from sklearn.preprocessing import Binarizer

In [107]:
pred_pro = model.predict_proba(X_train)[:, 1].reshape(-1,1) # 해약으로 분류 할 확률 가져와 한줄의 시리즈 형태로 변환 

In [109]:
# 해약으로 분류될 확률값을 가져와, 10%를 넘어가는 확률에 대해 -> 해약 / 10% 미만은 정상 
Y_train_proba = Binarizer(threshold=0.10).fit_transform(pred_pro)

In [112]:
print(classification_report(Y_train.replace({'정상':0, '해약':1}),
                            Y_train_proba))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     30106
           1       0.26      0.55      0.36       379

    accuracy                           0.98     30485
   macro avg       0.63      0.77      0.67     30485
weighted avg       0.99      0.98      0.98     30485

